### RapidFire AI Tutorial Use Case: SFT for Customer Support Q&A Chatbot

In [1]:
from rapidfireai import Experiment
from rapidfireai.automl import List, RFGridSearch, RFModelConfig, RFLoraConfig, RFSFTConfig

INFO 09-16 08:01:31 [__init__.py:241] Automatically detected platform cuda.


### Load Dataset and Specify Train and Eval Partitions

In [2]:
from datasets import load_dataset

dataset=load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

# Select a subset of the dataset for demo purposes
train_dataset=dataset["train"].select(range(5000))
eval_dataset=dataset["train"].select(range(5000,5200))
train_dataset=train_dataset.shuffle(seed=42)
eval_dataset=eval_dataset.shuffle(seed=42)

### Define Data Processing Function

In [3]:
def sample_formatting_function(row):
    """Function to preprocess each example from dataset"""
    # Special tokens for formatting
    SYSTEM_PROMPT = "You are a helpful and friendly customer support assistant. Please answer the user's query to the best of your ability."
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": row["instruction"]},
            
        ],
        "completion": [
            {"role": "assistant", "content": row["response"]}
        ]
    }

### Initialize Experiment

In [4]:
# Every experiment instance must be uniquely named
experiment = Experiment(experiment_name="exp1-chatqa")

2025/09/16 08:01:33 INFO mlflow.tracking.fluent: Experiment with name 'exp1-chatqa_7' does not exist. Creating a new experiment.


An experiment with the same name already exists. Created a new experiment with name 'exp1-chatqa_7' with Experiment ID: 12 and MLFlow Experiment ID: 12 saved at /home/palebluedot/test/rapidfireai/tutorial_notebooks/rapidfire_experiments/exp1-chatqa_7


### Define Custom Eval Metrics Function

In [5]:
def sample_compute_metrics(eval_preds):  
    """Optional function to compute eval metrics based on predictions and labels"""
    predictions, labels = eval_preds

    # Standard text-based eval metrics: Rouge and BLEU
    import evaluate
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")

    rouge_output = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)
    rouge_l = rouge_output["rougeL"]
    bleu_output = bleu.compute(predictions=predictions, references=labels)
    bleu_score = bleu_output["bleu"]

    return {
        "rougeL": round(rouge_l, 4),
        "bleu": round(bleu_score, 4),
    }

### Define Multi-Config Knobs for Model, LoRA, and SFT Trainer using RapidFire AI Wrapper APIs

In [6]:
# 2 LoRA PEFT configs with different adapter capacities
peft_configs = List([
    RFLoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        bias="none"
    ),
    RFLoraConfig(
        r=128,
        lora_alpha=256,
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        bias="none"
    )
])

# 2 base models x 2 peft configs = 4 combinations in total
config_set = List([
    RFModelConfig(
        model_name="rapidfire-ai-inc/Llama-3.1-8B-bnb-4bit",
        peft_config=peft_configs,
        training_args=RFSFTConfig(
            learning_rate=2e-4,
            lr_scheduler_type="linear",
            per_device_train_batch_size=4,
            per_device_eval_batch_size=8,
            num_train_epochs=2,
            gradient_accumulation_steps=4,
            logging_steps=5,
            eval_strategy="steps",
            eval_steps=25,
            fp16=True,
            save_strategy="epoch"
        ),
        model_type="causal_lm",
        model_kwargs={"load_in_4bit": True, "device_map": "auto", "torch_dtype": "auto", "use_cache": False},
        formatting_func = sample_formatting_function,
        compute_metrics = sample_compute_metrics,
        generation_config = { # This is for text based evaluation/prediction for causal_lm models
            "max_new_tokens": 256,
            "temperature": 0.6,
            "top_p": 0.9,
            "top_k": 40,
            "repetition_penalty": 1.18,
        }
    ),
    RFModelConfig(
        model_name="rapidfire-ai-inc/Mistral-7B-Instruct-v0.3-bnb-4bit",
        peft_config=peft_configs,
        training_args=RFSFTConfig(
            learning_rate=2e-4,
            lr_scheduler_type="linear",
            per_device_train_batch_size=4,
            per_device_eval_batch_size=8,
            num_train_epochs=2,
            gradient_accumulation_steps=4,
            logging_steps=5,
            eval_strategy="steps",
            eval_steps=25,
            fp16=True,
            save_strategy="epoch"
        ),
        model_type="causal_lm",
        model_kwargs={"load_in_4bit": True, "device_map": "auto", "torch_dtype": "auto", "use_cache": False},
        formatting_func = sample_formatting_function,
        compute_metrics = sample_compute_metrics,
        generation_config = { # This is for text based evaluation/prediction for causal_lm models
            "max_new_tokens": 256,
            "temperature": 0.6,
            "top_p": 0.9,
            "top_k": 40,
            "repetition_penalty": 1.18,
        }
    )
])


#### Define Model Creation Function for All Model Types Across Configs

In [7]:

def sample_create_model(model_config): 
     """Function to create model object for any given config; must return tuple of (model, tokenizer)"""
     from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM

     model_name = model_config["model_name"]
     model_type = model_config["model_type"]
     model_kwargs = model_config["model_kwargs"]
 
     if model_type == "causal_lm":
          model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "seq2seq_lm":
          model = AutoModelForSeq2SeqLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "masked_lm":
          model = AutoModelForMaskedLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "custom":
          # Handle custom model loading logic, e.g., loading your own checkpoints
          # model = ... 
          pass
     else:
          # Default to causal LM
          model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
      
     tokenizer = AutoTokenizer.from_pretrained(model_name)
      
     return (model,tokenizer)


#### Generate Config Group

In [8]:
# Simple grid search across all sets of config knob values = 4 combinations in total
config_group = RFGridSearch(
    configs=config_set,
    trainer_type="SFT"
)

### Run Multi-Config Training

In [9]:
# Launch training of all configs in the config_group with swap granularity of 4 chunks
experiment.run_fit(config_group, sample_create_model, train_dataset, eval_dataset, num_chunks=4, seed=42)

Started 4 worker processes successfully
Created workers
INFO 09-16 08:01:39 [__init__.py:241] Automatically detected platform cuda.
INFO 09-16 08:01:39 [__init__.py:241] Automatically detected platform cuda.
INFO 09-16 08:01:39 [__init__.py:241] Automatically detected platform cuda.
INFO 09-16 08:01:39 [__init__.py:241] Automatically detected platform cuda.
INFO 09-16 08:01:49 [__init__.py:241] Automatically detected platform cuda.
Run 3 has failed: element 0 of tensors does not require grad and does not have a grad_fnTraceback (most recent call last):
  File "/home/palebluedot/miniconda3/envs/test/lib/python3.12/site-packages/rapidfireai/backend/worker.py", line 240, in serve_forever
    self.run_fit(run_id, chunk_id, create_model_fn)
  File "/home/palebluedot/miniconda3/envs/test/lib/python3.12/site-packages/rapidfireai/backend/worker.py", line 159, in run_fit
    trainer_instance.train()
  File "/home/palebluedot/miniconda3/envs/test/lib/python3.12/site-packages/transformers/trainer

### End Current Experiment

In [10]:
experiment.end()

No active MLflow run to clear
Experiment exp1-chatqa_7 ended
Workers stopped
